In [41]:
import pandas as pd
import numpy as np

# data = pd.read_excel('../data/master.xlsx')
# data = pd.read_excel('../data/master.xlsx').sample(1000)
data = {'A': [3, 3, 2], 'B': [3, 3, 4], 'C': [7, 7, 7]}
data = pd.DataFrame(data)

minSup = 1
numDims = len(data.columns)
outputRec = pd.DataFrame({
    "Combination": [],
    "Count": []
})


In [42]:
data.columns

Index(['A', 'B', 'C'], dtype='object')

In [43]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [44]:
# unique_counts = data.nunique()
# col_dict = dict(zip(unique_counts.index, unique_counts.values))
# sorted_cols = sorted(col_dict, key=col_dict.get, reverse=True)
# data = data[sorted_cols]

In [45]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [46]:
def Partition(input_df, dim, start_row, end_row, dataCount):
    subset_df = input_df.iloc[start_row:end_row + 1, :]
    sorted_df = subset_df.sort_values(by=input_df.columns[dim])
    input_df.iloc[start_row:end_row + 1, :] = sorted_df.values
    counts = sorted_df.iloc[:, dim].value_counts().sort_index().tolist()
    dataCount.extend(counts)


In [ ]:
import heapq

def Partition(file_name, dim, start_row, end_row, dataCount):
    pageSize = 1000
    # read the file
    skipCount = start_row - 1
    if start_row == 0:
        skipCount = 0

    tempFileNames = []
        
    rows_read = 0
    pageIndex = -1

    while rows_read < end_row - start_row + 1:
        readCount = pageSize
        if rows_read + pageSize > end_row - start_row + 1:
            readCount = end_row - start_row + 1 - rows_read

        df = pd.read_excel(file_name, skiprows=skipCount, nrows=readCount, header=None)
        skipCount += readCount
        rows_read += readCount
        pageIndex += 1

        # sort the file
        df = df.sort_values(by=df.columns[dim])

        # write the file
        tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
        df.to_excel(tempFileName, index=False, header=False)

        # add the file name to the list
        tempFileNames.append(tempFileName)

    min_heap = []

    # Open all the temporary files
    file_handles = [open(tempFileName, 'r') for tempFileName in tempFileNames]

    # open  our original file
    original_file = open(file_name, 'r+')

    # go to the start row
    original_file.seek(start_row)

    # Initialize the heap with the first row from each file
    for i, file_handle in enumerate(file_handles):
        row = pd.read_excel(file_handle, nrows=1, header=None)
        heapq.heappush(min_heap, (row.iloc[0, dim], i, row))
        file_handle.seek(1)

    dataCountIndex = 0
    prev_value = None

    # while the heap is not empty
    while len(min_heap) > 0:
        value, file_index, row = heapq.heappop(min_heap)
        original_file.write(row.to_string(header=False, index=False))
        original_file.write("\n")

        if prev_value is None:
            prev_value = value
            dataCount.append(1)
        elif prev_value == value:
            dataCount[dataCountIndex] += 1
        else:
            dataCountIndex += 1
            dataCount.append(1)
            prev_value = value

        # read the next row from the file of index file_index if it is not empty
        row = pd.read_excel(file_handles[file_index], nrows=1, header=None)
        if not row.empty:
            heapq.heappush(min_heap, (row.iloc[0, dim], file_index, row))
            file_handles[file_index].seek(1)

In [47]:
def BottomUpCube(input, dim, start_row, end_row, currentCombination, numDims, minSup):
    global outputRec
    for d in range(dim, numDims):
        dataCount = []
        Partition(input, d, start_row, end_row, dataCount)

        k = start_row
        for c in dataCount:
            if c >= minSup:
                outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{
                    "Combination": currentCombination + [input.iloc[k, d]], 
                    "Count": c
                    }])], ignore_index=True)

                BottomUpCube(input, d + 1, k, k + c - 1, currentCombination + [input.iloc[k, d]], numDims, minSup)

            k += c    

In [48]:
count = len(data)
if count >= minSup:
    outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{"Combination": [], "Count": count}])], ignore_index=True)

In [49]:
BottomUpCube(data, 0, 0, len(data) - 1, [], numDims, minSup)

In [50]:
print(outputRec)

   Combination  Count
0           []    3.0
1          [2]    1.0
2       [2, 4]    1.0
3    [2, 4, 7]    1.0
4       [2, 7]    1.0
5          [3]    2.0
6       [3, 3]    2.0
7    [3, 3, 7]    2.0
8       [3, 7]    2.0
9          [3]    2.0
10      [3, 7]    2.0
11         [4]    1.0
12      [4, 7]    1.0
13         [7]    3.0


In [51]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7
